In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [71]:
from numpy.random import RandomState

In [49]:
np.random.seed(42)

In [73]:
RandomState(5).permutation(14)

array([ 5,  1,  7,  2, 10, 13, 11,  4,  8,  9,  0, 12,  6,  3])

In [3]:
CV_Match = pd.read_excel("./data/kaggle/data.xlsx", sheet_name= "Match", index_col = 'Candidate_match') 
CV_Sourcing = pd.read_excel("./data/kaggle/data.xlsx", sheet_name = "Candidate", index_col = 'CV_ID')

In [32]:
CV_Sourcing

,Experience,Language Level - French
CV_ID,,
1000,NaN,NaN
1001,NaN,NaN
1002,NaN,NaN
1003,NaN,NaN
1004,NaN,NaN
...,...,...
995,NaN,NaN
996,NaN,NaN
997,NaN,NaN


Adding in the data augmentation steps as outlined here: https://docs.google.com/document/d/13W0YIM3UcS--UGs5K5s5_DHYr0cfRZC_QYr146-3AfI/edit#heading=h.4rkzkuexnsde

In [88]:
CV_Sourcing.Experience.notnull().sum()

786

In [7]:
CV_Sourcing['Language Level - French'].unique()

array([nan, 'B1', 'A1', 'C2', 'Native', 'B2', 'A2', 'C1'], dtype=object)

In [37]:
dfMerged = CV_Sourcing.merge(CV_Match, left_on=CV_Sourcing.index, right_on=CV_Match.index).rename(columns={'key_0':'CV_ID'}).sort_index()

In [44]:
statusMap = {
'A01 - Offer Preparation Started':1,
 'A03 - Offer accepted':1,
 'A03 - Offer declined':1,
 'A03 - Process Cancelled':1,
 'A04 - Hiring Request Started':1,
 'A04b – Hiring Started / Contract signed':1,
 'A05 - Onboarding':1,
 'Approved':0,
 'CV approved':0,
 'CV dropped out':0,
 'CV refused':0,
 'CV sent to France':0,
 'Candidate dropped out':0,
 'Candidate refused':0,
 'Candidate validated':1,
 'D01 – Resignation':1,
 'Dropped out':0,
 'Matched':0,
 'Rejected':0,
 'Sent to Client':0,
 'T01- Terminated':1
}

In [45]:
dfMerged['MatchStatus'] = dfMerged['Status'].map(statusMap).astype(int)

In [47]:
dfMerged.MatchStatus.value_counts()

0    1908
1     347
Name: MatchStatus, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split

In [269]:
dfMerged

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,1
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,1
...,...,...,...,...,...,...,...,...,...
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,0
2253,9836,[0-1],NaN,12443,A05 - Onboarding,2017-12-01,2017-12-01,2017-12-01,1


In [230]:
dfMerged.iloc[:,-1]

0       1
1       0
2       0
3       0
4       1
       ..
2250    0
2251    0
2252    0
2253    1
2254    0
Name: MatchStatus, Length: 2255, dtype: int32

In [76]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMerged.iloc[:,:-1], dfMerged.iloc[:,-1], random_state=42, 
                                                test_size=.5, stratify=dfMerged.iloc[:,-1])

In [79]:
Xtrain.join(ytrain)['MatchStatus'].value_counts()

0    954
1    173
Name: MatchStatus, dtype: int64

In [80]:
Xtest.join(ytest)['MatchStatus'].value_counts()

0    954
1    174
Name: MatchStatus, dtype: int64

In [82]:
Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [96]:
baseGenderDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [224]:
# Deviations from demographic parity 
def deviation_func_gender(x, df, colName='Gender'):
    yesDF = df[(df.MatchStatus == 1)]
    noDF = df[~(df.MatchStatus == 1)]
    totalMalePercent = .5 + (x/100)
    totalMales = int(len(yesDF) * (totalMalePercent))
    
    np.random.seed(42)
    indM = np.random.choice(yesDF.index, totalMales, replace=False)
    indF = yesDF[~yesDF.index.isin(indM)].index
    
    yesDF.loc[indM, colName] = 'M'
    yesDF.loc[indF, colName] = 'F'
    
    dfNew = pd.concat([yesDF, noDF])
    return dfNew

In [292]:
# Deviations from demographic parity 
def deviation_func_race(x, df, colName='Black'):
    yesDF = df[(df.MatchStatus == 1)]
    noDF = df[(df.MatchStatus == 0)]
    totalBlackPercent = .5 + (x/100)
    totalBlacks = int(len(yesDF) * (totalBlackPercent))
    
    np.random.seed(42)
    indBlacks = np.random.choice(yesDF.index, totalBlacks, replace=False)
    indWhites = yesDF[~yesDF.index.isin(indBlacks)].index
    
    yesDF.loc[indBlacks, colName] = 0
    yesDF.loc[indWhites, colName] = 1
    
    dfNew = pd.concat([yesDF, noDF])
    return dfNew

In [238]:
deviation0 = deviation_func_gender(0, baseGenderDF)
deviation2 = deviation_func_gender(2, baseGenderDF)
deviation4 = deviation_func_gender(4, baseGenderDF)
deviation6 = deviation_func_gender(6, baseGenderDF)
deviation8 = deviation_func_gender(8, baseGenderDF)
deviation10 = deviation_func_gender(10, baseGenderDF)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [241]:
deviation0.to_csv('./data/kaggle/base-gender-dataset.csv')
deviation2.to_csv('./data/kaggle/base-gender-2-percent-dataset.csv')
deviation4.to_csv('./data/kaggle/base-gender-4-percent-dataset.csv')
deviation6.to_csv('./data/kaggle/base-gender-6-percent-dataset.csv')
deviation8.to_csv('./data/kaggle/base-gender-8-percent-dataset.csv')
deviation10.to_csv('./data/kaggle/base-gender-10-percent-dataset.csv')

In [266]:
baseGenderDF = baseGenderDF.drop('Gender', axis=1)

In [273]:
# Xtrain.drop('Gender', axis=1, inplace=True)
# Xtrain['Black'] = 1
Xtest.drop('Gender', axis=1, inplace=True)
Xtest['Black'] = 0

In [274]:
baseRaceDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [276]:
baseRaceDF.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        174         171                      133       174   
            1        173         169                      137       173   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         174                   174   
            1         173                   173   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    174   
            1                                    173   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                174  
            1                                173

In [293]:
racedeviation0 = deviation_func_race(0, baseRaceDF, colName='Black')
racedeviation2 = deviation_func_race(2, baseRaceDF)
racedeviation4 = deviation_func_race(4, baseRaceDF)
racedeviation6 = deviation_func_race(6, baseRaceDF)
racedeviation8 = deviation_func_race(8, baseRaceDF)
racedeviation10 = deviation_func_race(10, baseRaceDF)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [296]:
racedeviation0.to_csv('./data/kaggle/base-race-dataset.csv')
racedeviation2.to_csv('./data/kaggle/base-race-2-percent-dataset.csv')
racedeviation4.to_csv('./data/kaggle/base-race-4-percent-dataset.csv')
racedeviation6.to_csv('./data/kaggle/base-race-6-percent-dataset.csv')
racedeviation8.to_csv('./data/kaggle/base-race-8-percent-dataset.csv')
racedeviation10.to_csv('./data/kaggle/base-race-10-percent-dataset.csv')

In [294]:
racedeviation0.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        173         168                      135       173   
            1        174         172                      135       174   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         173                   173   
            1         174                   174   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    173   
            1                                    174   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                173  
            1                                174

In [295]:
racedeviation10.groupby(['MatchStatus','Black']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Black                                                         
0           0        954         205                      158       954   
            1        954         205                      172       954   
1           0        208         203                      165       208   
            1        139         137                      105       139   

                   Status  4. CV sent to Client  \
MatchStatus Black                                 
0           0         954                   650   
            1         954                   629   
1           0         208                   208   
            1         139                   139   

                   4. Client initial validation date  \
MatchStatus Black                                      
0           0                                    670   
            1                                    659   
1           0                                    208   
            1                                    139   

                   7. Client final approval date  
MatchStatus Black                                 
0           0                                279  
            1                                265  
1           0                                208  
            1                                139

In [240]:
deviation2.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0           F         954         205                      172       954   
            M         954         205                      158       954   
1           F         167         165                      129       167   
            M         180         175                      141       180   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0           F          954                   629   
            M          954                   650   
1           F          167                   167   
            M          180                   180   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0           F                                     659   
            M                                     670   
1           F                                     167   
            M                                     180   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                 265  
            M                                 279  
1           F                                 167  
            M                                 180

In [ ]:
# Rough Work

In [110]:
# CV_Sourcing['Gender'] = np.random.choice(['M','F'], 10099, p=[0.65, 0.35])

In [112]:
# CV_Sourcing['Race'] = np.random.choice(['American Indian or Alaska Native','Asian',
#                                        'Black or African American','Native Hawaiian or Other Pacific Islander',
#                                         'White', 'NA'], 10099, p=[0.05, 0.3, .18, .07, .35, .05])

In [116]:
# CV_Sourcing['Parent'] = np.random.choice([1,0], 10099, p=[0.30, 0.7])

In [129]:
# CV_Sourcing['City'] = np.random.choice(['Atlanta, GA','San Francisco, CA',
#                                        'Chicago, IL','New York City, NY',
#                                         'Houston, TX', 'Dallas, TX', 
#                                         'Salt Lake City, UT', 'Miami, FL',
#                                        'Los Angeles, CA','Seattle, WA'], 10099, 
#                                        p=[0.07, 0.18, .09, .11, .07, .1, .04, .12, .1, .12])

In [131]:
# CV_Sourcing['Occupation'] = np.random.choice(['Software Developer','Administrative Assistant',
#                                        'Customer Service','Cook',
#                                         'High-skilled Sales', 'Low-skilled Sales'], 10099, 
#                                        p=[0.43, 0.12, .2, .12, .1, .03])

In [134]:
CV_Sourcing.to_excel('./data/kaggle/augmented_data.xlsx')